---
title: UP scraper
author: Shalaka Shinde
date: 'August 12, 2023'
format:
    html:
        code-fold: true
---

(THIS CELL IS FOR THE QUARTO PAGE. DO NOT DELETE ON A WHIM.)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
#from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from tqdm.notebook import tqdm
import time
import pdb
import csv
from PIL import Image
import requests
import asyncio
import os
playwright = await async_playwright().start()
import re

In [2]:
# THIS LINE OPENS & SETS UP THE BROWSER
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [3]:
url = 'https://services.ecourts.gov.in/ecourtindia_v6/?p=casestatus'

In [11]:
# THIS LINE LOADS THE PAGE
await page.goto(url)

<Response url='https://services.ecourts.gov.in/ecourtindia_v6/?p=casestatus' request=<Request url='https://services.ecourts.gov.in/ecourtindia_v6/?p=casestatus' method='GET'>>

In [12]:
# THIS LINE REMOVES THE VALIDATION ERROR which asks you to select state, district and court complex
await page.locator('#validateError button').click()

In [13]:
# THIS CODE GETS THE LIST OF ALL THE STATE OPTIONS AVAILABLE 
state_options = page.locator("#sess_state_code option")
print(state_options)

<Locator frame=<Frame name= url='https://services.ecourts.gov.in/ecourtindia_v6/?p=casestatus&app_token=878e1f5cef762b7fa82e596fcd4dc8912805c8a638f58a3200b6884a5d819d56'> selector='#sess_state_code option'>


In [14]:
state_count = await state_options.count()
print(state_count)
check = []#["", "43", "15", "35", "17", "49", "44", "45", "7", "21", "67", "39", "36", "2", "46", "22", "47", "18", "75", "5", "41", "25", "33", "63", "9", "29", "20", "8", "28","50", "3", "64", "73", "4", "65", "70", "12", "66", "69", "51", "19", "40", "37", "34", "10", "23", "24", "52", "38", "14", "53", "42", "6", "68", "54", "26", "13", "31", "1", "55", "56", "57", "74", "71", "58", "76", "11", "59"]


38


In [8]:
async def get_districts():
    print('im here')
    #get the list of all district option for the selecged state
    district_options = page.locator("#sess_dist_code option")
    district_count = await district_options.count()
    print(district_count)
    print(district_options)
    for district_i in range(district_count):
                #get the locator for each district
                district_option = district_options.nth(district_i)
                print(district_option)
                #get the name
                district_text = await district_option.text_content()
                #format name of state to lowercase and replace space with underscore character
                district_name_for_filename = district_text.lower().strip().replace(" ", "_")
                time.sleep(3)
                print(district_name_for_filename)
                #get the district code
                district_value = await district_option.get_attribute('value')
                print(district_value)
                
                #again let's check by substring and not district code 
                if any(substring in district_name_for_filename for substring in ['select', 'select_district']):
                    print('Select State found')
                    continue
                else: 
                    #return the district_value list here so we can navigate districts in the main loop and not over here everytime 
                    return(district_value)

In [9]:
async def get_court_complexs():
    complex_options = page.locator("#court_complex_code option")
    complex_count = await complex_options.count()
    court_complex_names = []
    court_complex_codes = []

    time.sleep(2)
    print('im in courts function')

    for complex_i in range(complex_count):
        #get all court options 
        complex_option = complex_options.nth(complex_i)
        #get court name
        complex_text = await complex_option.text_content()
        #format name of court to lowercase and replace space with underscore character
        court_complex_name = complex_text.lower().strip().replace(" ", "_")
        #get court code
        print(court_complex_name)
        complex_value = await complex_option.get_attribute('value')
        print(complex_value)
        if any(substring in court_complex_name for substring in ["select_court_complex", "select"]):
            continue  # this will skip the "select court complex" option
        else: 
            court_complex_names.append(court_complex_name)
            court_complex_codes.append(complex_value)
    
    return court_complex_names, court_complex_codes


In [10]:
await page.locator('//*[@id="act-tabMenu"]').click()
await page.locator('#validateError button').click()
time.sleep(2)

TimeoutError: Timeout 30000ms exceeded.

In [15]:
for state_i in range(state_count):
        #get the locator frame for the states by the state number
        state_option = state_options.nth(state_i)
        #get the number of each state from the playwright locator 
        state_value = await state_option.get_attribute('value')
        #get the name of each state collected from the state number above
        state_text = await state_option.text_content()
        #format name of state to lowercase and replace space with underscore character
        state_name_for_filename = state_text.lower().strip().replace(" ", "_")
        time.sleep(3)
        
        #This has deprecated - now the Select State option is 0. To fix this, check for presence of substring. 
        if any(substring in state_text for substring in ['select', 'state', 'select_state']):
                print('Select State found')
                continue
        else:
                print("State found let's navigate")
                # Following line fills in the state option 
                await page.locator('#sess_state_code').select_option(state_value)
                time.sleep(1.5)
                #now let's collect the district data within each state:
                district_value = await get_districts() 
                print("district filled in: ", district_value)
                if district_value: 
                        #let's fill in the district value 
                        #fill in the district value
                        time.sleep(2)
                        await page.locator('#sess_dist_code').select_option(district_value)
                        time.sleep(3)
                        
                        court_names, court_codes = await get_court_complexs()
                        if court_codes != []:
                                for idx, i in enumerate(court_codes):
                                        #fill in the court code:    
                                        print("court code filled: ", i)     
                                        await page.locator('#court_complex_code').select_option(i)
                                        time.sleep(2)
                                        
                                        court_name = court_names[idx]
                                        print(court_name)
                                        # now let's navigate to the ACT tab to filter by IPC ACT 
                                        
                                        # THIS LINE REMOVES THE VALIDATION ERROR which asks you to select state, district and court complex
                                        await page.locator('//*[@id="act-tabMenu"]').click()
                                        #await page.locator('#validateError button').click()
                                        time.sleep(2)
                                        
                                        print('here here here')
                                        # FILL IN THE ACT TYPE 
                                        #Make sure that the Act Type dropdown is not empty like in Sitapur's Biswan Court Complex
                                        act_options = page.locator("#actcode option")
                                        print(act_options)
                                        act_count = await act_options.count()
                                        if act_count == 1:
                                                continue #Skip this turn if dropdown only has "Select Act Type" option WHY?!?!?!?
                                        
                                         # Look for "Indian Penal Code" or "IPC" or "I.P.C." (case-insensitive search)
                                        options = page.locator('#actcode').get_by_text(re.compile(r'\bIndian Penal Code', re.IGNORECASE))
                                        print('option: ', options)
                                        all_codes = options.all_text_contents()
                                        print("All codes: ", all_codes)
                                        #act_value = await .get_attribute('value')
                                        #print('act_val: ', act_value)
                                        
                                        break
        break
                
        
                

Select State found
State found let's navigate
im here
2
<Locator frame=<Frame name= url='https://services.ecourts.gov.in/ecourtindia_v6/?p=casestatus&app_token=878e1f5cef762b7fa82e596fcd4dc8912805c8a638f58a3200b6884a5d819d56'> selector='#sess_dist_code option'>
<Locator frame=<Frame name= url='https://services.ecourts.gov.in/ecourtindia_v6/?p=casestatus&app_token=878e1f5cef762b7fa82e596fcd4dc8912805c8a638f58a3200b6884a5d819d56'> selector='#sess_dist_code option >> nth=0'>
select_district

Select State found
<Locator frame=<Frame name= url='https://services.ecourts.gov.in/ecourtindia_v6/?p=casestatus&app_token=878e1f5cef762b7fa82e596fcd4dc8912805c8a638f58a3200b6884a5d819d56'> selector='#sess_dist_code option >> nth=1'>
port_blair
1
district filled in:  1
im in courts function
select_court_complex

port_blair_court_complex
1280004@2,3,4,5@N
court code filled:  1280004@2,3,4,5@N
port_blair_court_complex
here here here
<Locator frame=<Frame name= url='https://services.ecourts.gov.in/ecourt

In [116]:
async def main_hoops(state_options):

    list_of_htmls = []
    state_count = await state_options.count()
    check = []#["", "43", "15", "35", "17", "49", "44", "45", "7", "21", "67", "39", "36", "2", "46", "22", "47", "18", "75", "5", "41", "25", "33", "63", "9", "29", "20", "8", "28","50", "3", "64", "73", "4", "65", "70", "12", "66", "69", "51", "19", "40", "37", "34", "10", "23", "24", "52", "38", "14", "53", "42", "6", "68", "54", "26", "13", "31", "1", "55", "56", "57", "74", "71", "58", "76", "11", "59"]

    for state_i in range(state_count):
        #get the locator frame for the states by the state number
        state_option = state_options.nth(state_i)
        #get the number of each state from the playwright locator 
        state_value = await state_option.get_attribute('value')
        #get the name of each state collected from the state number above
        state_text = await state_option.text_content()
        #format name of state to lowercase and replace space with underscore character
        state_name_for_filename = state_text.lower().strip().replace(" ", "_")
        time.sleep(3)
        
        
        #This has deprecated - now the Select State option is 0. To fix this, check for presence of substring. 
        
        if state_text.str.contains("|".join(['select', 'state', 'select_state'])):
            print('Select State found')
            continue
        #if state_value == "13":
        #continue  # this will skip the "select state" option

        # Following line is navigating to each state
        await page.locator('#sess_state_code').select_option(state_value)

        time.sleep(1.5)

            district_options = page.locator("#sess_dist_code option")
            district_count = await district_options.count()
            
            for district_i in range(district_count):
                district_option = district_options.nth(district_i)
                district_text = await district_option.text_content()
                district_value = await district_option.get_attribute('value')
                
#                 print(f"{type(district_value)}")
                
                if district_value == "":
                    continue  # this will skip the "select district" option
                    
                time.sleep(2)
                await page.locator('#sess_dist_code').select_option(district_value)
                time.sleep(3)

                complex_options = page.locator("#court_complex_code option")
                complex_count = await complex_options.count()

                time.sleep(2)

                for complex_i in range(complex_count):
                    complex_option = complex_options.nth(complex_i)
                    complex_text = await complex_option.text_content()
                    complex_value = await complex_option.get_attribute('value')

                    if complex_value == "" or complex_text == "Select court complex":
                        continue  # this will skip the "select court complex" option

                    await page.locator('#court_complex_code').select_option(complex_value)
                    time.sleep(2)
                    await page.locator('//*[@id="act-tabMenu"]').click()
                    time.sleep(2)

                    file_name = f"{state_name_for_filename}_{complex_text.lower().replace(' ', '_')}.html"
                    #To avoid repeat scraping, if this file exists, then continue
                    directory = ("/Users/shalakashinde/Columbia/master_project/trial_2_court_page_htmls")
                    files = os.listdir(directory)
                    if file_name in files:
                        continue
                    
                    #Make sure that the Act Type dropdown is not empty like in Sitapur's Biswan Court Complex
                    act_options = page.locator("#actcode option")
                    act_count = await act_options.count()
                    if act_count == 1:
                        continue #Skip this turn if dropdown only has "Select Act Type" option

                    # Look for "Indian Penal Code" or "IPC" or "I.P.C." (case-insensitive search)
                    option = page.locator('#actcode').get_by_text(
                                            re.compile(r'\bIndian Penal Code\b|\bI\.P\.C\.\b|\bIPC\b', re.IGNORECASE))
                    act_value = await option.get_attribute('value')
                    
                    await page.locator('#actcode').select_option(act_value)

                    time.sleep(2.5)
                    await page.locator('#under_sec').fill('302')
                    time.sleep(2)
                    await page.locator('#radPAct').click()
                    print(f"I have hit page for {complex_text} in {district_value} which is {district_text} in {state_value} which is {state_text}")                        
                    time.sleep(3.5)
                    captcha = input('enter captcha here: ')

                    await page.locator('#act_captcha_code').fill(captcha)

                    await page.locator('#frm_act > div:nth-child(5) > div.col-md-auto > button').click()
                    await page.wait_for_load_state()
                        # extract html of the page
                    page_content = await page.content()
                        # append the extracted html content in the html file created above
                    with open(f"trial_2_court_page_htmls/{file_name}", 'a') as fp:
                        print(f"Saved as {file_name}")
                        fp.write(page_content + '<div class="dist_name">' + district_text + "</div>")

                    #append the district_code of the scraped district to the list "check"
                    #this will prevent duplication and reduce effort in the next round
                    check.append(district_value)

                    # append each html file to the list of htmls
                    list_of_htmls.append(file_name)

        else:
            continue

    return list_of_htmls

## STEP 1 Putting the function to use. BE CAREFUL WHILE DEPLOYING THIS CELL!
The next cell:
* reads into the state list from the browser to provide an input to the function
* the function runs and saves the output to a list (list_of_htmls)
* it tries to print all the htmls. Don't let it. That's a stupid step. But I'm keeping it.

In [117]:
state_options = page.locator("#sess_state_code option")
list_of_htmls = await main_hoops(state_options)
for each in list_of_htmls:
    print(each)

# table_1 = await page.query_selector_all('tr')
# for rows in table_1:
#     each_row = await rows.text_content()
#     print(each_row)

I have hit page for Railway Court, Agra in 30 which is Agra in 13 which is Uttar Pradesh
enter captcha here: tb86nv
Saved as uttar_pradesh_railway_court,_agra.html
I have hit page for Agra District Court Complex in 30 which is Agra in 13 which is Uttar Pradesh
enter captcha here: sb95rr
Saved as uttar_pradesh_agra_district_court_complex.html
I have hit page for CJJD, Fatehabad in 30 which is Agra in 13 which is Uttar Pradesh
enter captcha here: pstwcc
Saved as uttar_pradesh_cjjd,_fatehabad.html
I have hit page for Commercial Court Agra in 30 which is Agra in 13 which is Uttar Pradesh
enter captcha here: vytv5u
Saved as uttar_pradesh_commercial_court_agra.html
I have hit page for Iglas Court Complex in 43 which is Aligarh in 13 which is Uttar Pradesh
enter captcha here: l8bg6l
Saved as uttar_pradesh_iglas_court_complex.html
I have hit page for ACJM, Railway Court, Aligarh in 43 which is Aligarh in 13 which is Uttar Pradesh
enter captcha here: 2z9zys
Saved as uttar_pradesh_acjm,_railway_

CancelledError: 

## Overview of what came out of the first step
The next cell will:
* access the folder with all the htmls and turn it into a path
* take the html titles and turn them into a list called files
* this list had 179 elements for me
* Each list element is a single court's page
* Each page has a list of cases that the court's presiding judge is handling

In [1]:
# directory = ("/Users/shalakashinde/Columbia/master_project/court_page_htmls")
# files = os.listdir(directory)
# len(files)

## STEP 2 The next cell will give a single list of all CNRs from files that have data in it 
To do this, it will:
* access the local folder which has the raw scraped data files
* create an empty list
* read each file and check if there is a table tag in it
* if there is a table tag in a file, that file is a case of interest and append it to the empty list
* if there isn't one, it'll not append

In [9]:
import os
import pandas as pd
from bs4 import BeautifulSoup

# Initialize an empty list to store DataFrames
all_CNR = []

# Iterate over the HTML files
for file_name in files:
    # Read the file content
    directory = "/Users/shalakashinde/Columbia/master_project/court_page_htmls"
    file_path = os.path.join(directory, file_name)
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
    except:
        continue

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all "table" tags
    table_tags = soup.find_all('table')

    if table_tags:
        for each_table in table_tags:
            pretty_table = each_table.prettify()
#             print(pretty_table)
            soup_doc = BeautifulSoup(html_content, 'html.parser')

            # Find all elements with class "someclass"
            elements_with_someclass = soup_doc.find_all('a', class_='someclass')
            for element in elements_with_someclass:
                onclick_value = element.get('onclick')
                data_values = onclick_value.split("'")[1:-1]  # This will give you a list of data values
                CNR_number = data_values[0]
                all_CNR.append(CNR_number)
            
            
else:
    print("No tables found in the HTML files.")


No tables found in the HTML files.


In [ ]:
len(all_CNR) #Check the number of CNRs

### STEP 3 Save unique IDs in a csv file for safekeeping

In [129]:
safety_df = pd.DataFrame(all_CNR)
safety_df.to_csv('initial_scrapes.csv', index=False)

In [4]:
safety_df_1 = pd.read_csv('initial_scrapes.csv')
safety_df_1

,0
0,UPFZ010066362017
1,UPFZ010051922020
2,UPFZ010040312020
3,UPFZ010039322020
4,UPFZ010039312020
...,...
38462,UPBU040048182020
38463,UPBU040035572020
38464,UPBU040002402020
38465,UPSH180016292003


In [5]:
all_CNR = safety_df_1["0"].values.tolist()

**STEP 4.A** 

This is normal processing of API requests made to the undocumented API of the website. It is mainly for reference. Takes a lot of time to make single requests. Can be used as a reference point for setting up parallel processing.

## STEP 4.A Single thread processing. 
This cell will make one call to the API at a time, copy the contents on the API output to an HTML file, and save the HTML file.

In [ ]:


cookies = {
    'PHPSESSID': '14ff51gchi7uo3a46tflb8ilul',
    'PHPSESSID': 'fneluai8og9b7osal4h8304n4i',
}

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': 'PHPSESSID=14ff51gchi7uo3a46tflb8ilul; PHPSESSID=fneluai8og9b7osal4h8304n4i',
    'Origin': 'https://services.ecourts.gov.in',
    'Referer': 'https://services.ecourts.gov.in/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}
# timeout_seconds = 10
directory = ("/Users/shalakashinde/Columbia/master_project/each_case_htmls")
files = os.listdir(directory)
for each_CNR in tqdm(all_CNR):

    filename = f"{each_CNR}.html"
#     import pdb
#     pdb.set_trace()
    if filename in files:
        continue
    
    data = {
        'court_code': '15',
        'state_code': '1',
        'dist_code': '23',
        'court_complex_code': '1010261',
        'case_no': '200205009811992',
        'cino': each_CNR,
        'hideparty': '',
        'search_flag': 'CScaseNumber',
        'search_by': 'CSact',
        'ajax_req': 'true',
        'app_token': '0263f592ee4f1ea3a4ee93b95f5bc6f299ac500fedd5340a64003b5e1bf8b571',
    }

    response = requests.post(
        'https://services.ecourts.gov.in/ecourtindia_v6/?p=home/viewHistory',
    #     cookies=cookies,
        headers=headers,
        data=data,
        
    )
    
    #Strip the html_code to make it ready for the next step
    try:
        # Attempt to access the 'data_list' key in the JSON response
        html_code = response.json()['data_list'].strip()
        
        #This step will go to the folder which stores each case file, open the html file, and add the response.json() to it
        #This will ensure that the API is not hit at all if this case file already exists
        with open(f"each_case_htmls/{filename}", 'a') as fp:
            
            print(f"Saved as {filename}")
            fp.write(html_code)
    except KeyError:
        # If 'data_list' key is not present, print the entire JSON response for debugging
        print(len(files))
        print(f"Error: 'data_list' key not found in the JSON response: {response.json()}")
        continue

#     time.sleep(1.5)
#     case_dict = extract_dates_from_API_response(html_code)
#     if case_dict in list_of_dicts_of_all_cases:
#         continue
#     else:
#         list_of_dicts_of_all_cases.append(case_dict)
    
    
    
    #Next step is to run it through BeautifulSoup and extracting dates
    #each case's dates will be one dict 
    #each dict will be appended to a larger list which will eventually turn into a dataframe

In [20]:
print(response.text)

{"data_list":"<h2 class='h4 text-center  mb-1' tabindex='0' id='chHeading'>District and Session Judge<\/h2>\n\t\t\t<h3 class='h2class fw-bold text-center'>Case Details<\/h3><table class=\"table case_details_table table-bordered\">\n\t\t\t\t\t\t<tbody>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td><label class=\"fw-bold\"> <\/label>Case Type<\/td>\n\t\t\t\t\t\t<td colspan='3'  class=\"fw-bold text-uppercase\">Misc. \/12<\/td>\n\t\t\t\t\t<\/tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td><label class=\"fw-bold\"> Filing Number <\/label><\/td>\n\t\t\t\t\t\t<td class=\"fw-bold\">571\/2016 &nbsp;<\/td>\n\t\t\t\t\t\t<td><label class=\"fw-bold\">Filing Date<\/label><\/td>\n\t\t\t\t\t\t<td class=\"fw-bold\">  09-02-2016  &nbsp;<\/td>\n\t\t\t\t\t<\/tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td><label class=\"fw-bold\">Registration Number<\/label><\/td>\n\t\t\t\t\t\t<td><label style=\"font-weight:bold;\">26\/2016<\/label><\/td>\n\t\t\t\t\t\t<td><label style=\"font-weight:bold;\">Registration Date:<\/label><\/td>\n\t\t\t\t\t\t<t

34971
Error: 'data_list' key not found in the JSON response: {'errormsg': 'THERE IS AN ERROR', 'app_token': '552419c7f4dc8e40bd9764b82ce7442d7a4ea6c606f773037247cb2075c6a916'}
34971
Error: 'data_list' key not found in the JSON response: {'errormsg': 'THERE IS AN ERROR', 'app_token': '7a323400dc979598f8187601bb509064f98a1b00a22af91d6f73afe8db8ec69a'}


**STEP 4.B**

This is the step that will use the API from STEP 4.A and set up parallel processing using Thread Pool Executor. Make this more robust. It reduces time by A LOT! BUT, learn how threads are activated. 

## STEP 4.B Thread Pool Executor method 
Following cell is the function that will do the same process as above but it makes 6 calls at the same time instead of just one.

In [34]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

directory = ("/Users/shalakashinde/Columbia/master_project/each_case_htmls")
files = os.listdir(directory)

def process_cnr(each_CNR):
    
    filename = f"{each_CNR}.html"
    if filename in files:
        return
    
    data = {
        'court_code': '15',
        'state_code': '1',
        'dist_code': '23',
        'court_complex_code': '1010261',
        'case_no': '200205009811992',
        'cino': each_CNR,
        'hideparty': '',
        'search_flag': 'CScaseNumber',
        'search_by': 'CSact',
        'ajax_req': 'true',
        'app_token': 'cd0a44642bdcc5ac4e0110f7478cfa38f546b508a3c2eec8b48c6f62689898ac',
    }

    
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        # 'Cookie': 'PHPSESSID=14ff51gchi7uo3a46tflb8ilul; PHPSESSID=fneluai8og9b7osal4h8304n4i',
        'Origin': 'https://services.ecourts.gov.in',
        'Referer': 'https://services.ecourts.gov.in/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
    }

    response = requests.post(
        'https://services.ecourts.gov.in/ecourtindia_v6/?p=home/viewHistory',
        headers=headers,
        data=data,
    )

    # Strip the html_code to make it ready for the next step
    try:
        # Attempt to access the 'data_list' key in the JSON response
        html_code = response.json()['data_list'].strip()

        # This step will go to the folder which stores each case file, open the html file, and add the response.json() to it
        # This will ensure that the API is not hit at all if this case file already exists
        with open(f"each_case_htmls/{filename}", 'a') as fp:
            print(f"Saved as {filename}")
            fp.write(html_code)
    except KeyError:
        # If 'data_list' key is not present, print the entire JSON response for debugging
        print(len(files))
        print(f"Error: 'data_list' key not found in the JSON response: {response.json()}")
        return

In [ ]:
# 'all_CNR' is defined after the Playwright scraper in the cells above

# Maximum number of threads to use
max_threads = 6

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    # Submit each CNR to the executor for processing
    futures = [executor.submit(process_cnr, cnr) for cnr in tqdm(all_CNR)]

# Wait for all futures to complete (optional but recommended)
print("It has reached here")
for future in tqdm(futures):
    future.result()


In [ ]:
print(response.text)